# Глубинное обучение для текстовых данных, ФКН ВШЭ

## Домашнее задание 2: Рекуррентные нейронные сети

### Оценивание и штрафы


Максимально допустимая оценка за работу — __10 (+3) баллов__. Сдавать задание после указанного срока сдачи нельзя.

Задание выполняется самостоятельно. «Похожие» решения считаются плагиатом и все задействованные студенты (в том числе те, у кого списали) не могут получить за него больше 0 баллов. Весь код должен быть написан самостоятельно. Чужим кодом для пользоваться запрещается даже с указанием ссылки на источник. В разумных рамках, конечно. Взять пару очевидных строчек кода для реализации какого-то небольшого функционала можно.

Неэффективная реализация кода может негативно отразиться на оценке. Также оценка может быть снижена за плохо читаемый код и плохо оформленные графики. Все ответы должны сопровождаться кодом или комментариями о том, как они были получены.

__Мягкий дедлайн: 5.10.25 23:59__   
__Жесткий дедлайн: 8.10.25 23:59__


### О задании

In [17]:
import sys
sys.version

'3.10.18 (main, Oct  6 2025, 22:29:25) [GCC 15.2.1 20250813]'

В этом задании вам предстоит самостоятельно реализовать модель LSTM для решения задачи классификации с пересекающимися классами (multi-label classification). Это вид классификации, в которой каждый объект может относиться одновременно к нескольким классам. Такая задача часто возникает при классификации фильмов по жанрам, научных или новостных статей по темам, музыкальных композиций по инструментам и так далее.

В нашем случае мы будем работать с датасетом биотехнических новостей и классифицировать их по темам. Этот датасет уже предобработан: текст приведен к нижнему регистру, удалена пунктуация, все слова разделены проблелом.

In [18]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [19]:
import pandas as pd

dataset = pd.read_csv('data/biotech_news.tsv', sep='\t')
dataset.sample(5)

,text,labels
2103,email rotterdam a former discount supermarket ...,expanding geography
1837,updated 48 mins ago matt cardy getty images fr...,"regulatory approval, executive statement"
1385,health global ophthalmology devices market ass...,company description
1923,saturday june 15 2019 young entrepreneur joins...,participation in an event
299,share on pinterest share on linkedin escanaba ...,other


In [20]:
dataset.loc[dataset.labels.str.contains('other')].labels.value_counts()

labels
other    484
Name: count, dtype: int64

будем считать, что other это не категория, а отсутствие категории

## Предобработка лейблов


__Задание 1 (0.5 балла)__. Как вы можете заметить, лейблы записаны в виде строк, разделенных запятыми. Для работы с ними нам нужно преобразовать их в числа. Так как каждый объект может принадлежать нескольким классам, закодируйте лейблы в виде векторов из 0 и 1, где 1 означает, что объект принадлежит соответствующему классу, а 0 – не принадлежит. Имея такую кодировку, мы сможем обучить модель, решая задачу бинарной классификации для каждого класса.

In [22]:
res = set()
for s in dataset.labels:
    labels = s.split(', ')
    res = res | set(labels)
keys = sorted(list(res))
keys.remove('other')
labels = dict(zip(keys, range(len(keys))))
labels

{'alliance & partnership': 0,
 'article publication': 1,
 'clinical trial sponsorship': 2,
 'closing': 3,
 'company description': 4,
 'department establishment': 5,
 'event organization': 6,
 'executive appointment': 7,
 'executive statement': 8,
 'expanding geography': 9,
 'expanding industry': 10,
 'foundation': 11,
 'funding round': 12,
 'hiring': 13,
 'investment in public company': 14,
 'ipo exit': 15,
 'm&a': 16,
 'new initiatives & programs': 17,
 'new initiatives or programs': 18,
 'participation in an event': 19,
 'partnerships & alliances': 20,
 'patent publication': 21,
 'product launching & presentation': 22,
 'product updates': 23,
 'regulatory approval': 24,
 'service & product providing': 25,
 'subsidiary establishment': 26,
 'support & philanthropy': 27}

In [23]:
dataset = pd.read_csv('data/biotech_news.tsv', sep='\t')

res = set()
for s in dataset.labels:
    labels = s.split(', ')
    res = res | set(labels)
keys = sorted(list(res))
keys.remove('other')
labels = dict(zip(keys, range(len(keys))))

In [26]:
    def encode_labels(labels_str):
        if labels_str == 'other':
            return np.zeros(len(keys), dtype=np.float32)

        encoded = np.zeros(len(keys), dtype=np.float32)
        for label in labels_str.split(', '):
            if label in labels:
                encoded[labels[label]] = 1.0
        return encoded

encode_labels('alliance & partnership, closing')

array([1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)

In [27]:
dataset['encoded_labels'] = dataset.labels.apply(encode_labels)

## Предобработка данных

В этом задании мы будем обучать рекуррентные нейронные сети. Как вы знаете, они работают лучше для коротких текстов, так как не очень хорошо улавливают далекие зависимости. Для уменьшение длин текстов их стоит почистить.

Сразу разделим выборку на обучающую и тестовую, чтобы считать все нужные статистики только по обучающей.

In [28]:
from sklearn.model_selection import train_test_split

texts_train, texts_test, y_train, y_test = train_test_split(
    dataset.text.values,
    dataset.encoded_labels.values,
    test_size=0.2,  # do not change this
    random_state=0  # do not change this
)

__Задание 2 (1 балл)__. Удалите из текстов стоп слова, слишком редкие и слишком частые слова. Гиперпараметры подберите самостоятельно (в идеале их стоит подбирать по качеству на тестовой выборке). Если вы считаете, что стоит добавить еще какую-то обработку, то сделайте это. Важно не удалить ничего, что может повлиять на предсказание класса.

СДЕЛАНО В dataset.py

In [32]:
from gensim.parsing.preprocessing import preprocess_string, strip_tags, strip_punctuation, strip_multiple_whitespaces, strip_numeric, remove_stopwords
from nltk.stem import PorterStemmer


stemmer = PorterStemmer()

def clean(txt: str):
    CUSTOM_FILTERS = [
        strip_tags,
        strip_punctuation,
        strip_multiple_whitespaces,
        strip_numeric,
        remove_stopwords,
    ]
    res = preprocess_string(txt, CUSTOM_FILTERS)
    res = (stemmer.stem(word) for word in res)
    return ' '.join(res)

dataset['preprocessed'] = dataset.text.apply(clean)

In [35]:
dataset[['preprocessed', 'labels']]

,preprocessed,labels
0,drive plow bone dead olga tokarczuk incred slo...,other
1,recent tabl nation budget denel alloc r billio...,other
2,share break good pictur getti metro uk tempt t...,other
3,reso current hire posit product managerand mem...,other
4,charter buyer club charter buyer club charter ...,other
...,...,...
3034,publish hour ago grate famili present chequ de...,"funding round, support & philanthropy, executi..."
3035,cenexelcent excel join nearli clinic research ...,clinical trial sponsorship
3036,jun m pt repli respons ongo covid pandem famil...,"new initiatives or programs, funding round, ex..."
3037,whatsapp photo suppli red river wast solut for...,"service & product providing, closing, company ..."


In [37]:
dataset.to_csv('data/preprocessed.csv', index=False)

__Задание 3 (1.5 балла)__. Осталось перевести тексты в индексы токенов, чтобы их можно было подавать в модель. У вас есть две опции, как это сделать:
1. __(+0 баллов)__ Токенизировать тексты по словам.
2. __(до +3 баллов)__ Реализовать свою токенизацию BPE. Количество баллов будет варьироваться в зависимости от эффективности реализации. При реализации нельзя пользоваться специализированными библиотеками.

Токенизируйте тексты, переведите их в списки индексов и сложите вместе с лейблами в `DataLoader`. Не забудьте добавить в `DataLoader` `collate_fn`, которая будет дополнять все короткие тексты в батче паддингами. Для маппинга токенов в индексы вам может пригодиться `gensim.corpora.dictionary.Dictionary`.

In [20]:
from vocab import Vocab
from dataset import TextDataset, collate

from torch.utils.data import DataLoader

vocab = Vocab(texts_train, 2, 0.7)

device = 'cuda:0'

train_dataset = TextDataset(texts_train, y_train, vocab, device)
test_dataset = TextDataset(texts_test, y_test, vocab, device)

train_loader = DataLoader(
    train_dataset, 
    batch_size=32, 
    shuffle=True,
    collate_fn=lambda batch: collate(batch, vocab.pad_idx)
)

val_loader = DataLoader(
    test_dataset, 
    batch_size=128, 
    shuffle=False,
    collate_fn=lambda batch: collate(batch, vocab.pad_idx)
)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 608/608 [00:00<00:00, 47966.46it/s]


In [9]:
len(vocab.dictionary)

14331

In [10]:
for batch_texts, batch_labels in train_loader:
    print(f"Texts shape: {batch_texts.shape}")
    print(f"Labels shape: {batch_labels.shape}")
    print(f"Labels dtype: {batch_labels.dtype}")
    break

Texts shape: torch.Size([32, 655])
Labels shape: torch.Size([32, 29])
Labels dtype: torch.float32


## Метрика качества

Перед тем, как приступить к обучению, нам нужно выбрать метрику оценки качества. Так как в задаче классификации с пересекающимися классами классы часто несбалансированы, чаще всего в качестве метрики берется [F1 score](https://en.wikipedia.org/wiki/F-score).

Функция `compute_f1` принимает истинные метки и предсказанные и считает среднее значение F1 по всем классам. Используйте ее для оценки качества моделей.

$$
F1_{total} = \frac{1}{K} \sum_{k=1}^K F1(Y_k, \hat{Y}_k),
$$
где $Y_k$ – истинные значения для класса k, а $\hat{Y}_k$ – предсказания.

In [11]:
from sklearn.metrics import f1_score

def compute_f1(y_true, y_pred):
    assert y_true.ndim == 2
    assert y_true.shape == y_pred.shape

    return f1_score(y_true, y_pred, average='macro')

## Обучение моделей

### RNN

В качестве бейзлайна обучим самую простую рекуррентную нейронную сеть. Напомним, что блок RNN выглядит таким образом.

<img src="https://i.postimg.cc/yYbNBm6G/tg-image-1635618906.png" alt="drawing" width="400"/>

Его скрытое состояние обновляется по формуле
$h_t = \sigma(W x_{t} + U h_{t-1} + b_h)$. А предсказание считается с помощью применения линейного слоя к последнему токену
$o_T = V h_T + b_o$. В качестве функции активации выберите гиперболический тангенс. 

__Задание 4 (2 балла)__. Реализуйте RNN в соответствии с формулой выше и обучите ее на нашу задачу. Нулевой скрытый вектор инициализируйте нулями, так модель будет обучаться стабильнее, чем при случайной инициализации. После этого замеряйте качество на тестовой выборке. У вас должно получиться значение F1 не меньше 0.33, а само обучение не должно занимать много времени.

In [5]:
from sklearn.model_selection import train_test_split

import torch
from torch.optim import AdamW, lr_scheduler
from torch.nn import BCEWithLogitsLoss
from torch.utils.data import DataLoader

import numpy as np

from rnn import RNN
from deep import LSTM
from torchrnn import TorchRNN
from vocab import Vocab
from vocab_bpe import BPEVocab
from dataset import TextDataset, collate, load_biotech, load_preprocessed
from train import train

In [15]:
BATCH_SIZE = 256

WARMUP_EPOCHS = 12
EPOCHS = 72
LR = 3e-3
WD = 1e-3

FACTOR = 0.5
PATIENCE = 5

config = {
    'embed_dim': 128,
    'hidden_size': 256,
    'num_classes': 28,
    'dropout_rate_emb': 0.1,
    'dropout_rate_linear': 0.5,
    'dropout_rate_hidden': 0.1,
    # 'num_layers': 2
}


In [16]:
print('Loading dataset...')
# dataset, labels = load_biotech()
dataset, labels = load_preprocessed()

device = 'cuda:0'
texts_train, texts_test, y_train, y_test = train_test_split(
    dataset.text.values,
    dataset.encoded_labels.values,
    test_size=0.2,  # do not change this
    random_state=0  # do not change this
)

print('Building vocab...')
vocab = Vocab(texts_train, no_below=26, no_above=0.68, max_tokens=36, clean=False)
# vocab = BPEVocab(texts_train, vocab_size=3000, max_tokens=36, clean=False)

print('Vocab size:', len(vocab.dictionary))

print('Tokenizing ...')
train_dataset = TextDataset(texts_train, y_train, vocab, device)
test_dataset = TextDataset(texts_test, y_test, vocab, device)

train_loader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    collate_fn=lambda batch: collate(batch, vocab.pad_idx)
)

val_loader = DataLoader(
    test_dataset,
    batch_size=128,
    shuffle=False,
    collate_fn=lambda batch: collate(batch, vocab.pad_idx)
)

Loading dataset...
Building vocab...


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2431/2431 [00:00<00:00, 3044.82it/s]


Vocab size: 3646
Tokenizing ...


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 608/608 [00:00<00:00, 47645.63it/s]


In [17]:
class_weights = torch.tensor((y_train.shape[0] - y_train.sum(0)) / (y_train.sum(0) + 1e-6), dtype=torch.float32, device=device) # type: ignore
# class_weights = torch.tensor((y_train.shape[0]) / (y_train.sum(0) + 1e-6), dtype=torch.float32, device=device)

config['vocab_size'] = len(vocab.dictionary)
config['pad_idx'] = vocab.pad_idx

gt = torch.stack([torch.tensor(label) for label in y_test], dim=0).cpu()
model = RNN(**config).to(device)

train_config = {
    'warmup_epochs': WARMUP_EPOCHS,
    'epochs': EPOCHS,
    'model': model,
    'optim': AdamW(model.parameters(), lr=LR, weight_decay=WD),
    'criterion': BCEWithLogitsLoss(pos_weight=class_weights),
    'train_loader': train_loader,
    'val_loader': val_loader,
    'val_gt': gt,
}


total_warmup_steps = WARMUP_EPOCHS * len(train_loader) 

warmup_scheduler = lr_scheduler.LinearLR(
    train_config['optim'],
    start_factor=0.01,
    end_factor=1.0,
    total_iters=total_warmup_steps
)


plateau_scheduler = lr_scheduler.ReduceLROnPlateau(
    train_config['optim'],
    mode='max',
    factor=0.5,
    patience=3,
    min_lr=1e-7,
)

train_config['warmup_scheduler'] = warmup_scheduler
train_config['plateau_scheduler'] = plateau_scheduler

print('Training model...')
train(**train_config)

Training model...
│ Epoch 01 │ Train Loss: 1.3742 │ Val Loss: 1.3961 │ F1: 8.69% │ LR: 0.00027750 │
│ Epoch 02 │ Train Loss: 1.3500 │ Val Loss: 1.3825 │ F1: 8.70% │ LR: 0.00052500 │
│ Epoch 03 │ Train Loss: 1.3205 │ Val Loss: 1.3776 │ F1: 8.49% │ LR: 0.00077250 │
│ Epoch 04 │ Train Loss: 1.2979 │ Val Loss: 1.4439 │ F1: 8.80% │ LR: 0.00102000 │
│ Epoch 05 │ Train Loss: 1.2656 │ Val Loss: 1.4130 │ F1: 8.45% │ LR: 0.00126750 │
│ Epoch 06 │ Train Loss: 1.2405 │ Val Loss: 1.3529 │ F1: 9.24% │ LR: 0.00151500 │
│ Epoch 07 │ Train Loss: 1.2315 │ Val Loss: 1.6068 │ F1: 8.24% │ LR: 0.00176250 │
│ Epoch 08 │ Train Loss: 1.1811 │ Val Loss: 1.5093 │ F1: 10.65% │ LR: 0.00201000 │
│ Epoch 09 │ Train Loss: 1.1700 │ Val Loss: 1.4442 │ F1: 9.88% │ LR: 0.00225750 │
│ Epoch 10 │ Train Loss: 1.1583 │ Val Loss: 1.4757 │ F1: 11.76% │ LR: 0.00250500 │
│ Epoch 11 │ Train Loss: 1.1030 │ Val Loss: 1.4144 │ F1: 10.94% │ LR: 0.00275250 │
│ Epoch 12 │ Train Loss: 1.1230 │ Val Loss: 1.3668 │ F1: 11.19% │ LR: 0.00300

### LSTM

<img src="https://i.postimg.cc/pL5LdmpL/tg-image-2290675322.png" alt="drawing" width="400"/>

Теперь перейдем к более продвинутым рекурренным моделям, а именно LSTM. Из-за дополнительного вектора памяти эта модель должна гораздо лучше улавливать далекие зависимости, что должно напрямую отражаться на качестве.

Параметры блока LSTM обновляются вот так ($\sigma$ означает сигмоиду):
\begin{align}
f_{t} &= \sigma(W_f x_{t} + U_f h_{t-1} + b_f) \\ 
i_{t} &= \sigma(W_i x_{t} + U_i h_{t-1} + b_i) \\
\tilde{c}_{t} &= \tanh(W_c x_{t} + U_c h_{t-1} + b_i) \\
c_{t} &= f_t \odot c_{t-1} + i_t \odot \tilde{c}_t \\
o_{t} &= \sigma(W_t x_{t} + U_t h_{t-1} + b_t) \\
h_t &= o_t \odot \tanh(c_t)
\end{align}

__Задание 5 (2 балла).__ Реализуйте LSTM по описанной схеме. Выберите гиперпараметры LSTM так, чтобы их общее число (без учета слоя эмбеддингов) примерно совпадало с числом параметров обычной RNN, но размерность скрытого слоя была не меньше 64. Так мы будем сравнивать архитектуры максимально независимо. Обучите LSTM до сходимости и сравните качество с RNN на тестовой выборке. Удалось ли получить лучший результат? Как вы можете это объяснить?

In [19]:
from lstm import LSTM

BATCH_SIZE = 256

WARMUP_EPOCHS = 12
EPOCHS = 72
LR = 3e-3
WD = 1e-3

FACTOR = 0.5
PATIENCE = 5

config = {
    'embed_dim': 128,
    'hidden_size': 256,
    'num_classes': 28,
    'dropout_rate_emb': 0.1,
    'dropout_rate_linear': 0.5,
    'dropout_rate_hidden': 0.1,
    # 'num_layers': 2
}

config['vocab_size'] = len(vocab.dictionary)
config['pad_idx'] = vocab.pad_idx

gt = torch.stack([torch.tensor(label) for label in y_test], dim=0).cpu()
model = LSTM(**config).to(device)

train_config = {
    'warmup_epochs': WARMUP_EPOCHS,
    'epochs': EPOCHS,
    'model': model,
    'optim': AdamW(model.parameters(), lr=LR, weight_decay=WD),
    'criterion': BCEWithLogitsLoss(pos_weight=class_weights),
    'train_loader': train_loader,
    'val_loader': val_loader,
    'val_gt': gt,
}


total_warmup_steps = WARMUP_EPOCHS * len(train_loader) 

warmup_scheduler = lr_scheduler.LinearLR(
    train_config['optim'],
    start_factor=0.01,
    end_factor=1.0,
    total_iters=total_warmup_steps
)


plateau_scheduler = lr_scheduler.ReduceLROnPlateau(
    train_config['optim'],
    mode='max',
    factor=0.5,
    patience=3,
    min_lr=1e-7,
)

train_config['warmup_scheduler'] = warmup_scheduler
train_config['plateau_scheduler'] = plateau_scheduler

print('Training model...')
train(**train_config)

Training model...
│ Epoch 01 │ Train Loss: 1.2922 │ Val Loss: 1.3910 │ F1: 8.43% │ LR: 0.00027750 │
│ Epoch 02 │ Train Loss: 1.3045 │ Val Loss: 1.3899 │ F1: 9.30% │ LR: 0.00052500 │
│ Epoch 03 │ Train Loss: 1.2907 │ Val Loss: 1.3879 │ F1: 10.08% │ LR: 0.00077250 │
│ Epoch 04 │ Train Loss: 1.2903 │ Val Loss: 1.3849 │ F1: 10.97% │ LR: 0.00102000 │
│ Epoch 05 │ Train Loss: 1.2639 │ Val Loss: 1.4865 │ F1: 9.11% │ LR: 0.00126750 │
│ Epoch 06 │ Train Loss: 1.2410 │ Val Loss: 1.3208 │ F1: 11.35% │ LR: 0.00151500 │
│ Epoch 07 │ Train Loss: 1.1548 │ Val Loss: 1.3142 │ F1: 13.66% │ LR: 0.00176250 │
│ Epoch 08 │ Train Loss: 1.1201 │ Val Loss: 1.3037 │ F1: 14.55% │ LR: 0.00201000 │
│ Epoch 09 │ Train Loss: 1.0852 │ Val Loss: 1.2112 │ F1: 13.57% │ LR: 0.00225750 │
│ Epoch 10 │ Train Loss: 1.0847 │ Val Loss: 1.4101 │ F1: 14.37% │ LR: 0.00250500 │
│ Epoch 11 │ Train Loss: 0.9826 │ Val Loss: 1.2518 │ F1: 15.16% │ LR: 0.00275250 │
│ Epoch 12 │ Train Loss: 0.9863 │ Val Loss: 1.3732 │ F1: 17.21% │ LR: 0.

__Задание 6 (2 балла).__ Главный недостаток RNN моделей заключается в том, что при сжатии всей информации в один вектор, важные детали пропадают. Для решения этой проблемы был придуман механизм внимания. Реализуйте его по [оригинальной статье](https://arxiv.org/abs/1409.0473). Замерьте качество и сделайте выводы.   
Обратите внимание, что метод был предложен для Encoder-Decoder моделей. В нашем случае декодера нет, поэтому встройте внимание в энкодер: каждый блок LSTM будет смотреть на выходы всех предыдущих блоков.   

In [20]:
from attention import LSTM

BATCH_SIZE = 256

WARMUP_EPOCHS = 12
EPOCHS = 72
LR = 3e-3
WD = 1e-3

FACTOR = 0.5
PATIENCE = 5

config = {
    'embed_dim': 128,
    'hidden_size': 256,
    'num_classes': 28,
    'dropout_rate_emb': 0.1,
    'dropout_rate_linear': 0.5,
    'dropout_rate_hidden': 0.1,
    # 'num_layers': 2
}

config['vocab_size'] = len(vocab.dictionary)
config['pad_idx'] = vocab.pad_idx

gt = torch.stack([torch.tensor(label) for label in y_test], dim=0).cpu()
model = LSTM(**config).to(device)

train_config = {
    'warmup_epochs': WARMUP_EPOCHS,
    'epochs': EPOCHS,
    'model': model,
    'optim': AdamW(model.parameters(), lr=LR, weight_decay=WD),
    'criterion': BCEWithLogitsLoss(pos_weight=class_weights),
    'train_loader': train_loader,
    'val_loader': val_loader,
    'val_gt': gt,
}


total_warmup_steps = WARMUP_EPOCHS * len(train_loader) 

warmup_scheduler = lr_scheduler.LinearLR(
    train_config['optim'],
    start_factor=0.01,
    end_factor=1.0,
    total_iters=total_warmup_steps
)


plateau_scheduler = lr_scheduler.ReduceLROnPlateau(
    train_config['optim'],
    mode='max',
    factor=0.5,
    patience=3,
    min_lr=1e-7,
)

train_config['warmup_scheduler'] = warmup_scheduler
train_config['plateau_scheduler'] = plateau_scheduler

print('Training model...')
train(**train_config)

Training model...
│ Epoch 01 │ Train Loss: 1.3001 │ Val Loss: 1.3925 │ F1: 9.40% │ LR: 0.00027750 │
│ Epoch 02 │ Train Loss: 1.3044 │ Val Loss: 1.3919 │ F1: 10.84% │ LR: 0.00052500 │
│ Epoch 03 │ Train Loss: 1.3004 │ Val Loss: 1.3899 │ F1: 11.29% │ LR: 0.00077250 │
│ Epoch 04 │ Train Loss: 1.2821 │ Val Loss: 1.5080 │ F1: 8.77% │ LR: 0.00102000 │
│ Epoch 05 │ Train Loss: 1.2336 │ Val Loss: 1.3939 │ F1: 11.02% │ LR: 0.00126750 │
│ Epoch 06 │ Train Loss: 1.1727 │ Val Loss: 1.5227 │ F1: 13.14% │ LR: 0.00151500 │
│ Epoch 07 │ Train Loss: 1.1366 │ Val Loss: 1.3618 │ F1: 12.22% │ LR: 0.00176250 │
│ Epoch 08 │ Train Loss: 1.1556 │ Val Loss: 1.5737 │ F1: 12.55% │ LR: 0.00201000 │
│ Epoch 09 │ Train Loss: 1.0867 │ Val Loss: 1.4545 │ F1: 13.00% │ LR: 0.00225750 │
│ Epoch 10 │ Train Loss: 1.0588 │ Val Loss: 1.7775 │ F1: 14.09% │ LR: 0.00250500 │
│ Epoch 11 │ Train Loss: 1.0500 │ Val Loss: 1.5078 │ F1: 14.85% │ LR: 0.00275250 │
│ Epoch 12 │ Train Loss: 0.9818 │ Val Loss: 1.5781 │ F1: 16.15% │ LR: 0

__Задание 7 (1 балл).__ Добавьте в вашу реализации возможность увеличивать число слоев LSTM. Обучите модель с двумя слоями и замерьте качество. Сделайте выводы: стоит ли увеличивать размер модели?

In [22]:
from deep import LSTM

BATCH_SIZE = 256

WARMUP_EPOCHS = 12
EPOCHS = 72
LR = 3e-3
WD = 1e-3

FACTOR = 0.5
PATIENCE = 5

config = {
    'embed_dim': 128,
    'hidden_size': 256,
    'num_classes': 28,
    'dropout_rate_emb': 0.1,
    'dropout_rate_linear': 0.5,
    'dropout_rate_hidden': 0.1,
    'num_layers': 2
}

config['vocab_size'] = len(vocab.dictionary)
config['pad_idx'] = vocab.pad_idx

gt = torch.stack([torch.tensor(label) for label in y_test], dim=0).cpu()
model = LSTM(**config).to(device)

train_config = {
    'warmup_epochs': WARMUP_EPOCHS,
    'epochs': EPOCHS,
    'model': model,
    'optim': AdamW(model.parameters(), lr=LR, weight_decay=WD),
    'criterion': BCEWithLogitsLoss(pos_weight=class_weights),
    'train_loader': train_loader,
    'val_loader': val_loader,
    'val_gt': gt,
}


total_warmup_steps = WARMUP_EPOCHS * len(train_loader) 

warmup_scheduler = lr_scheduler.LinearLR(
    train_config['optim'],
    start_factor=0.01,
    end_factor=1.0,
    total_iters=total_warmup_steps
)


plateau_scheduler = lr_scheduler.ReduceLROnPlateau(
    train_config['optim'],
    mode='max',
    factor=0.5,
    patience=3,
    min_lr=1e-7,
)

train_config['warmup_scheduler'] = warmup_scheduler
train_config['plateau_scheduler'] = plateau_scheduler

print('Training model...')
train(**train_config)

Training model...
│ Epoch 01 │ Train Loss: 1.2963 │ Val Loss: 1.3914 │ F1: 8.87% │ LR: 0.00027750 │
│ Epoch 02 │ Train Loss: 1.3213 │ Val Loss: 1.3915 │ F1: 9.70% │ LR: 0.00052500 │
│ Epoch 03 │ Train Loss: 1.3005 │ Val Loss: 1.3908 │ F1: 12.87% │ LR: 0.00077250 │
│ Epoch 04 │ Train Loss: 1.2832 │ Val Loss: 1.3659 │ F1: 11.34% │ LR: 0.00102000 │
│ Epoch 05 │ Train Loss: 1.2154 │ Val Loss: 1.2639 │ F1: 12.31% │ LR: 0.00126750 │
│ Epoch 06 │ Train Loss: 1.1704 │ Val Loss: 1.3678 │ F1: 12.40% │ LR: 0.00151500 │
│ Epoch 07 │ Train Loss: 1.1154 │ Val Loss: 1.3889 │ F1: 15.13% │ LR: 0.00176250 │
│ Epoch 08 │ Train Loss: 1.0845 │ Val Loss: 1.4447 │ F1: 13.34% │ LR: 0.00201000 │
│ Epoch 09 │ Train Loss: 1.0953 │ Val Loss: 1.3842 │ F1: 14.68% │ LR: 0.00225750 │
│ Epoch 10 │ Train Loss: 1.0755 │ Val Loss: 1.4109 │ F1: 13.21% │ LR: 0.00250500 │
│ Epoch 11 │ Train Loss: 1.0331 │ Val Loss: 1.3295 │ F1: 15.55% │ LR: 0.00275250 │
│ Epoch 12 │ Train Loss: 1.0234 │ Val Loss: 1.7213 │ F1: 17.38% │ LR: 0

# BPE with RUST

In [23]:
from sklearn.model_selection import train_test_split

import torch
from torch.optim import AdamW, lr_scheduler
from torch.nn import BCEWithLogitsLoss
from torch.utils.data import DataLoader

import numpy as np

from rnn import RNN
from deep import LSTM
from torchrnn import TorchRNN
# from vocab import Vocab
from vocab_bpe import BPEVocab
from dataset import TextDataset, collate, load_biotech, load_preprocessed
from train import train

BATCH_SIZE = 256

WARMUP_EPOCHS = 12
EPOCHS = 72
LR = 3e-3
WD = 1e-3

FACTOR = 0.5
PATIENCE = 5

config = {
    'embed_dim': 128,
    'hidden_size': 256,
    'num_classes': 28,
    'dropout_rate_emb': 0.1,
    'dropout_rate_linear': 0.5,
    'dropout_rate_hidden': 0.1,
    'num_layers': 2
}

print('Loading dataset...')
# dataset, labels = load_biotech()
dataset, labels = load_preprocessed()

device = 'cuda:0'
texts_train, texts_test, y_train, y_test = train_test_split(
    dataset.text.values,
    dataset.encoded_labels.values,
    test_size=0.2,  # do not change this
    random_state=0  # do not change this
)

print('Building vocab...')
# vocab = Vocab(texts_train, no_below=26, no_above=0.68, max_tokens=36, clean=False)
vocab = BPEVocab(texts_train, vocab_size=4000, max_tokens=100, clean=False)

# print('Vocab size:', len(vocab.dictionary))

print('Tokenizing ...')
train_dataset = TextDataset(texts_train, y_train, vocab, device)
test_dataset = TextDataset(texts_test, y_test, vocab, device)

train_loader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    collate_fn=lambda batch: collate(batch, vocab.pad_idx)
)

val_loader = DataLoader(
    test_dataset,
    batch_size=128,
    shuffle=False,
    collate_fn=lambda batch: collate(batch, vocab.pad_idx)
)

class_weights = torch.tensor((y_train.shape[0] - y_train.sum(0)) / (y_train.sum(0) + 1e-6), dtype=torch.float32, device=device) # type: ignore
# class_weights = torch.tensor((y_train.shape[0]) / (y_train.sum(0) + 1e-6), dtype=torch.float32, device=device)

config['vocab_size'] = vocab.tokenizer.vocab_size()
config['pad_idx'] = vocab.pad_idx

gt = torch.stack([torch.tensor(label) for label in y_test], dim=0).cpu()
model = LSTM(**config).to(device)

train_config = {
    'warmup_epochs': WARMUP_EPOCHS,
    'epochs': EPOCHS,
    'model': model,
    'optim': AdamW(model.parameters(), lr=LR, weight_decay=WD),
    'criterion': BCEWithLogitsLoss(pos_weight=class_weights),
    'train_loader': train_loader,
    'val_loader': val_loader,
    'val_gt': gt,
}


total_warmup_steps = WARMUP_EPOCHS * len(train_loader) 

warmup_scheduler = lr_scheduler.LinearLR(
    train_config['optim'],
    start_factor=0.01,
    end_factor=1.0,
    total_iters=total_warmup_steps
)


plateau_scheduler = lr_scheduler.ReduceLROnPlateau(
    train_config['optim'],
    mode='max',
    factor=0.5,
    patience=3,
    min_lr=1e-7,
)

train_config['warmup_scheduler'] = warmup_scheduler
train_config['plateau_scheduler'] = plateau_scheduler

print('Training model...')
train(**train_config)


Loading dataset...
Building vocab...


Preprocessing: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2431/2431 [00:00<00:00, 2235062.04it/s]


Tokenizing ...


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 608/608 [00:00<00:00, 55123.79it/s]


Training model...
│ Epoch 01 │ Train Loss: 1.2978 │ Val Loss: 1.3922 │ F1: 8.94% │ LR: 0.00027750 │
│ Epoch 02 │ Train Loss: 1.2960 │ Val Loss: 1.3898 │ F1: 9.92% │ LR: 0.00052500 │
│ Epoch 03 │ Train Loss: 1.2976 │ Val Loss: 1.3894 │ F1: 10.14% │ LR: 0.00077250 │
│ Epoch 04 │ Train Loss: 1.2784 │ Val Loss: 1.3492 │ F1: 10.52% │ LR: 0.00102000 │
│ Epoch 05 │ Train Loss: 1.2072 │ Val Loss: 1.3541 │ F1: 9.55% │ LR: 0.00126750 │
│ Epoch 06 │ Train Loss: 1.2093 │ Val Loss: 1.3664 │ F1: 11.96% │ LR: 0.00151500 │
│ Epoch 07 │ Train Loss: 1.1477 │ Val Loss: 1.3416 │ F1: 14.15% │ LR: 0.00176250 │
│ Epoch 08 │ Train Loss: 1.1012 │ Val Loss: 1.3409 │ F1: 11.91% │ LR: 0.00201000 │
│ Epoch 09 │ Train Loss: 1.0840 │ Val Loss: 1.4369 │ F1: 18.13% │ LR: 0.00225750 │
│ Epoch 10 │ Train Loss: 1.0413 │ Val Loss: 1.5317 │ F1: 13.42% │ LR: 0.00250500 │
│ Epoch 11 │ Train Loss: 1.0719 │ Val Loss: 1.2693 │ F1: 14.65% │ LR: 0.00275250 │
│ Epoch 12 │ Train Loss: 1.0736 │ Val Loss: 1.4570 │ F1: 16.12% │ LR: 0.